In [ ]:
import sys
IN_COLAB = 'google.colab' in sys.modules
IN_COLAB

In [ ]:
if IN_COLAB:
  !pip install python-dotenv

In [ ]:
from dotenv import load_dotenv, find_dotenv

In [ ]:
import json, textwrap

In [ ]:
load_dotenv('huit-at-test/.env')

In [ ]:
import os

In [ ]:
# if IN_COLAB:
#   from google.colab import userdata
#   os.environ['OPENAI_KEY'] = userdata.get('OPENAI_KEY')
#   os.environ['OPENAI_SECRET'] = userdata.get('OPENAI_SECRET')
if IN_COLAB:
  from getpass import getpass
  OPENAI_KEY = getpass("OPENAI_KEY")
  OPENAI_SECRET = getpass("OPENAI_SECRET")
else:
  OPENAI_KEY = os.getenv('OPENAI_KEY')
  OPENAI_SECRET = os.getenv('OPENAI_SECRET')

In [ ]:
harvard_api = 'https://go.apis.huit.harvard.edu/ais-openai-direct-limited-schools/v1'

In [ ]:
from openai import OpenAI, BaseModel
client = OpenAI(
    api_key=OPENAI_KEY,
    base_url=harvard_api
)

In [ ]:
raw_text = """
Of Man’s first disobedience, and the fruit
Of that forbidden tree whose mortal taste
Brought death into the World, and all our woe,
With loss of Eden, till one greater Man
Restore us, and regain the blissful seat,
Sing Heavenly Muse, that, on the secret top
Of Oreb or of Sinai, didst inspire
That shepherd who first taught the chosen seed,
In the beginning how the heavens and earth
Rose out of Chaos: or if Sion Hill
Delight thee more, and Siloa’s brook that flow’d
Fast by the oracle of God; I thence
Invoke thy aid to my advent’rous song,
That with no middle flight intends to soar
Above th’ Aonian mount, while it pursues
Things unattempted yet in prose or rhyme.
"""

class PLExcerpt(BaseModel):
    book: int
    lines: str
    characters: list[str]
    themes: list[str]
    summary: str

    model_config = {"extra": "forbid"}

schema = PLExcerpt.model_json_schema()

In [ ]:
response = client.responses.create(
    model="gpt-4o-2024-08-06",
    input=[
        {
            "role": "system",
            "content": "Extract structured data about the passage. Make sure the summary is concise and technical, fitting for an academic well-versed in the text. Remove any filler or aggrandizement."
                       "Return JSON only—no extra keys or text."
        },
        {
            "role": "user",
            "content": raw_text
        },
    ],
    text={
        "format": {
            "type": "json_schema",
            "name": "paradise_lost_excerpt",
            "strict": True,
            "schema": schema,
        }
    },
)

print("Parsed object:\n")

parsed = json.loads(response.output_text)
parsed["summary"] = textwrap.fill(parsed["summary"], width=80)
print(json.dumps(parsed, indent=2))

In [ ]:
import requests

In [ ]:
client.api_base = harvard_api

In [ ]:
url = "https://go.apis.huit.harvard.edu/ais-openai-direct-limited-schools/v1/chat/completions"
payload = json.dumps({
  "model": "gpt-4o-mini",
  "messages": [
    {
      "role": "user",
      "content": "To what extent does scientific conservatism limit new discoveries?"
    }
  ],
  "temperature": 0.7
})
headers = {
  'Content-Type': 'application/json',
  'api-key': OPENAI_KEY
}
response = requests.request("POST", url, headers=headers, data=payload)
print(response.text)

In [ ]:
response.json()